In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_2 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.2",  model=m12_6_2,
output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_2_data = Dict("N" => size(d, 1), "T" => d[:total_tools],
"N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_2_data, ProjDir,
diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Info

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE       MCSE         ESS    
         lp__ -39.585620650  3.1257106642 0.04942182503 0.1111624351  790.644848
accept_stat__   0.934363813  0.0902924453 0.00142764891 0.0019821972 1000.000000
   stepsize__   0.052165925  0.0066049346 0.00010443319 0.0010575038   39.009752
  treedepth__   5.893750000  0.4415990149 0.00698229350 0.0185523773  566.574539
 n_leapfrog__  72.480000000 32.4467686866 0.51302845881 1.0609913120  935.231767
  divergent__   0.000000000  0.0000000000 0.00000000000 0.0000000000         NaN
     energy__  46.040967375  4.0203899648 0.06356794685 0.1348216527  889.236064
        alpha   1.087590421  0.7712629351 0.01219473775 0.0213218790 1000.000000
  a_society.1  -0.202061054  0.2397064309 0.00379009146 0.0056674858 1000.000000
  a_society.2   0.044039516  0.2226497804 0.00352040213 0.0053824170 1000.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*